In [1]:
#conding:utf-8
import tensorflow as tf

INPUT_NODE = 784 #28*28像素点
OUTPUT_NODE = 10 #输出10个数
LAYER1_NODE = 500 #隐藏层节点个数

def get_weight(shape,regularizer):
    w = tf.Variable(tf.truncated_normal(shape,stddev=0.1))
    if regularizer != None:
        tf.add_to_collection('losses',tf.contrib.layers.l2_regularizer(regularizer)(w))
    return w

def get_bias(shape):
    b = tf.Variable(tf.zeros(shape))
    return b

def forward(x,regularizer):
    w1 = get_weight([INPUT_NODE,LAYER1_NODE],regularizer)
    b1 = get_bias([LAYER1_NODE])
    y1 = tf.nn.relu(tf.matmul(x,w1)+b1)
    
    w2 = get_weight([LAYER1_NODE,OUTPUT_NODE,],regularizer)
    b2 = get_bias([OUTPUT_NODE])
    y = tf.nn.relu(tf.matmul(y1,w2)+b2)
    
    return y

In [3]:
#conding:utf-8
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
#import mnist_forward

BATCH_SIZE = 200
LEARNING_RATE_BASE = 0.1
LEARNING_RATE_DECAY = 0.99
REGULARIZER = 0.0001
STEPS = 50000
MOVING_AVERAGE_DECAY = 0.99
MODEL_SAVE_PATH = "/Users/szkfzx/datasets/mnist/model"
MODEL_NAME = "mnist_model"

def backward(mnist):
    
    x = tf.placeholder(tf.float32,[None,INPUT_NODE])
    y_ = tf.placeholder(tf.float32,[None,OUTPUT_NODE])
    y = forward(x,REGULARIZER)
    global_step = tf.Variable(0,trainable=False)
    
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
    cem = tf.reduce_mean(ce)
    loss = cem + tf.add_n(tf.get_collection('losses'))
    
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE,
                                               global_step,
                                               mnist.train.num_examples / BATCH_SIZE,
                                               LEARNING_RATE_DECAY,
                                               staircase=True)
    
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,
                                                                           global_step=global_step)
    
    ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    ema_op = ema.apply(tf.trainable_variables())
    with tf.control_dependencies([train_step,ema_op]):
        train_op = tf.no_op(name='train')
        
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        init_op = tf.global_variables_initializer()
        sess.run(init_op)
        
        for i in range(STEPS):
            xs,ys = mnist.train.next_batch(BATCH_SIZE)
            _,loss_value,step = sess.run([train_op,loss,global_step],feed_dict={x:xs,y_:ys})
            if i % 1000 == 0:
                print "After %d training step(s),loss on training batch is %g" % (step,loss_value)
                saver.save(sess,os.path.join(MODEL_SAVE_PATH,MODEL_NAME),global_step=global_step)
                
def main():
    mnist = input_data.read_data_sets("/Users/szkfzx/datasets/mnist/data/",one_hot=True)
    backward(mnist)
    
if __name__ == "__main__":
    main()

Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/szkfzx/datasets/mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use urllib or similar directly.


IOError: [Errno socket error] [Errno 60] Operation timed out

In [3]:
#conding:utf-8
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time
# import mnist_forward
# import mnist_backward

TEST_INTERVAL_SECS = 5

def test(mnist):
    with tf.Graph().as_default() as g:
        x = tf.placeholder(tf.float32,[None,INPUT_NODE])
        y_ = tf.placeholder(tf.float32,[None,OUTPUT_NODE])
        y = forward(x,None)
    
        ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        ema_restore = ema.variables_to_restore()
        saver = tf.train.Saver(ema_restore)
        
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        
        while True:
            with tf.Session() as sess:
                ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
                if ckpt and ckpt.model_checkpoint_path:
                    saver.restore(sess,ckpt.model_checkpoint_path)
                    global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    accuracy_score = sess.run(accuracy,feed_dict={x:mnist.test.images,y_:mnist.test.labels})
                    print "After %s training step(s),test accuracy = %g" % (global_step,accuracy_score)
                else:
                    print "No checkpoint file found"
                    return
            time.sleep(TEST_INTERVAL_SECS)
            break
            
def main():
    mnist = input_data.read_data_sets("/Users/he/Desktop/hehuanlin/datasets/mnist/data/",one_hot=True)
    test(mnist)
    
if __name__ == '__main__':
    main()

Extracting /Users/he/Desktop/hehuanlin/datasets/mnist/data/train-images-idx3-ubyte.gz
Extracting /Users/he/Desktop/hehuanlin/datasets/mnist/data/train-labels-idx1-ubyte.gz
Extracting /Users/he/Desktop/hehuanlin/datasets/mnist/data/t10k-images-idx3-ubyte.gz
Extracting /Users/he/Desktop/hehuanlin/datasets/mnist/data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Restoring parameters from /Users/he/Desktop/hehuanlin/datasets/mnist/model/mnist_model-49001
After 49001 training step(s),test accuracy = 0.9799


In [4]:
#conding:utf-8
import tensorflow as tf
import numpy as np
from PIL import Image
import os

def restore_model(testPicArr):
    #创建一个默认图，在该图中执行以下操作（多数操作和train中一样，就不再重复解释，大家对照学习即可）
    with tf.Graph().as_default() as tg:
        x = tf.placeholder(tf.float32,[None,INPUT_NODE])
        y = forward(x,None)
        preValue = tf.argmax(y,1)#得到概率最大的预测值
        
        #实现滑动平均模型，参数MOVING_AVERAGE_DECAY用于控制模型更新的速度，训练过程中会对每一个变量维护一个影子变量
        #这个影子变量就是相应变量的初始值，每次变量更新时，影子变量就会随之更新
        variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        variables_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)

        with tf.Session() as sess:
            #通过checkpoint文件定位到最新保存的模型
            ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess,ckpt.model_checkpoint_path)
                
                preValue = sess.run(preValue,feed_dict={x:testPicArr})
                return preValue
            else:
                print "No checkpoint file found"
                return -1

#预处理函数，包括resize、转变灰度图、二值化操作 
def pre_dic(picName):
    img = Image.open(picName)
    reIm = img.resize((28,28),Image.ANTIALIAS)
    im_arr = np.array(reIm.convert('L'))
    threshold = 50#设定合理的阈值
    for i in range(28):
        for j in range(28):
            im_arr[i][j] = 255 - im_arr[i][j]
            if(im_arr[i][j] < threshold):
                im_arr[i][j] = 0
            else:
                im_arr[i][j] = 255
                
    nm_arr = im_arr.reshape([1,784])
    nm_arr = nm_arr.astype(np.float32)
    img_ready = np.multiply(nm_arr,1.0/255.0)
    
    return img_ready
            
def application():
    testNum = input("input the number of test pictures:") 
    for i in range(testNum):
        testPic = raw_input("the path of test picture:")
        testPicArr = pre_pic(testPic)
        preValue = restore_model(testPicArr)
        print "The prediction number is:", preValue
        
def main():
    application()
    
if __name__ == '__main()':
    main()

In [ ]:
#conding:utf-8
import tensorflow as tf
import numpy as np
from PIL import Image
import os

image_train_path = '/Users/he/Desktop/hehuanlin/datasets/mnist/data/mnist_data_jpg/mnist_train_jpg_60000/'
label_train_path = '/Users/he/Desktop/hehuanlin/datasets/mnist/data/mnist_data_jpg/mnist_train_jpg_60000.txt'
tfRecord_train = '/Users/he/Desktop/hehuanlin/datasets/mnist/data/mnist_train.tfrecords'
image_test_path = '/Users/he/Desktop/hehuanlin/datasets/mnist/data/mnist_data_jpg/mnist_test_jpg_10000/'
lebel_train_path = '/Users/he/Desktop/hehuanlin/datasets/mnist/data/mnist_data_jpg/mnist_test_jpg_10000.txt'
tfRecord_test = '/Users/he/Desktop/hehuanlin/datasets/mnist/data/mnist_test.tfrecords'
data_path = '/Users/he/Desktop/hehuanlin/datasets/mnist/data/'
resize_height = 28
resize_width = 28

def write_tfRecord(tfRecordName,image_path,label_path):
    writer = tf.python_io.TFRecordWriter(tfRecordName)
    num_pic = 0
    f = open(lebel_path,'r')
    f.close()
    for content in contents:
        value = content.split()
        img_path = image_path + value[0]
        img = Image.open(img_path)
        img_raw = img.tobytes()
        labels = [0] * 10
        lebels[int(value[1])] = 1
        
        example = tf.train.Example(features=tf.train.Features(feature={
            'img_raw':tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw])),
            'label':tf.train.Feature(int64_list=tf.train.Int64List(value=lebels))
        }))
        
        writer.write(example.SerializeToString())
        num_pic += 1
        print "the number of picture:",num_pic
        
    writer.close()
    print "write tfrecord successful"
    
def generate_tfRecord():
    isExists = os.path.exists(data_path)
    if not isExists:
        os.makedirs(data_path)
        print "The directory was created successfully"
    else:
        print "directory already exists"
    wirte_tfRecord(tfRecord_train,image_train_path,label_train_path)
    wirte_tfRecord(tfRecord_test,image_test_path,label_test_path)
    
def read_tfRecord(tfRecord_path):
    filename_queue = tf.train.string_input_producer([tfRecord_path])
    reader = tf.TFRecorderReader()
    _,serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(serialized_example,
                                      features={
                                          'label':tf.FixedLenFeature([10],tf.int64),
                                          'img_raw':tf.FixedLenFeature([],tf.string)
                                      })
    img = tf.decode_raw(features['img_raw'],tf.uint8)
    img.set_shape([784])
    img = tf.cast(img,tf.float32)*(1./255)
    label = tf.cast(features['label'],tf.float32)
    return img,label

def get_tfrecord(num,isTrain=True):
    if isTrain:
        tfRecord_path = tfRecord_train
    else:
        tfRecord_path = tfRecord_test
    img,lebel = read_tfRecord(tfRecord_path)
    img_batch,label_batch = tf.train.shuffle_batch([img,label],
                                                   batch_size=num,
                                                   num_threads=2,
                                                   capacity=1000,
                                                   min_after_dequeue=700)
    return img_batch,label_batch

def main():
    generate_tfRecord()
    
if __name__ == '__main__':
    main()